# **Single Fingerprint Deep Classification**
### **JCE - Software Engineering Final Project** 
##### ***By Kobi Amsellem & Zohar Kedem***

##### In this study we want to discovare if Deep Convolutional Neural Network can classify single fingerprint image to find the owner: 
##### **1. Gender** - 2 classes (male/female).
##### **2. Finger name** - 10 classes (right-thumb, ..., right-thumb, ...)
##### **3. Fingerprint type** - 5 classes (left loop, whirl, right loop, tented arch, arch)
##### **4. Same Person** - 2 classes (Same, Different) *(whether or not two fingerprints belong to the same personperson)* 


In [1]:
import os
import copy
import numpy as np
import datasets as ds
import tensorflow as tf
import FPMLmodule.utils as utils
import FPMLmodule.backbones as bb
import FPMLmodule.classifiers as cf
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from FPMLmodule.fpml import FPML 
from pathlib import Path

print('TensorFlow Version:', tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
dbStudyOutPath ='./out/{}/'

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

TensorFlow Version: 2.8.0


#### **Model Search** 
##### Main method to search after the best model

in this section we define the ````researchBestModel````, a method to search and compare between several different models and hyper parameters and choose the best one for specific dataset.

##### **Stages:**
##### **1.** Backbone Transfer Learning mode comparison - (a) Pretraind wheights and Untrainable, (b) Pretraind wheights and Trainable, (c) Initialize wheight and Trainable.
##### **2.** Backbone comparison. 
##### **3.** Classifier (perceptron) comparison. 
##### **4.** Loss function comparison. 
##### **5.** Learning rates comparison. 
##### **6.** Optimizers comparison. 
##### **7.** Train best configuration for extra epochs on train+validation datasets

In [5]:
def researchBestModel(configureDS, datasets, epochsSearch, epochsBest, path, optimizers, 
                      learningRates, defaultHypers, losses, classifiers, defaultClassifier, 
                      defaultBackbone, backbonesForSearch, verbose=1, top=1):
    trainDs, testDs, valDs = datasets
    imgDim = configureDS.inputDim
    nbClasses = len(configureDS.classNames)
    outFilePath = path + 'ablationHistory.csv'
    topModelsPath = path+'FinalModels/'
    
    print('Ablation for', configureDS.name)
    
    
    baseStudy = {
            "architecture": {
                "backbone": defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=False),
                "classifier": defaultClassifier(nbClasses, "softmax"),
                "inputLayer": None, 
                "inputDim": imgDim
            },
            "hyperparameters": defaultHypers
        }
    
    # --------------------------------------------------
    # ---- Stage 1 - Compare Transfer Learning Mode ----
    # --------------------------------------------------
    
    trainingModeStudy = baseStudy
    trainingModeStudy['architecture']['backbone'] = [
                    defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=False, name='PT&Untrainable'),
                    defaultBackbone['backbone'](imgDim, weights=defaultBackbone['weights'], trainable=True, name='PT&Trainable'),
                    defaultBackbone['backbone'](imgDim, weights=None, trainable=True, name='IN&Trainable')
                    ]    
    trainingModeHistories = utils.researchStudies(trainDs, valDs, {'DNN-TransferLearningMode' : trainingModeStudy}, epochsSearch, verbose)
    bestTrainMode = utils.getBestStudyFromHistories(trainingModeHistories)
    trainable, pretrained = 'Trainable' in bestTrainMode, 'PT' in bestTrainMode
    utils.displayStudiesProgress(trainingModeHistories, path, 'Comparison Transfer Learning Mode')
    utils.saveStudyHistory({'DNN-TransferLearningMode' : trainingModeStudy}, trainingModeHistories, outFilePath)
    print("Backbone Transfer Learning Mode Selected - Pretrained:", pretrained, 'Trainable:', trainable)
    
    
    # ------------------------------------------
    # ---- Stage 2 - Compare DNN Backbones -----
    # ------------------------------------------
    backboneConfigs = [backbone['backbone'](imgDim, weights=backbone['weights'], trainable=trainable) for backbone in backbonesForSearch]
    bestBackboneStudy = copy.deepcopy(trainingModeHistories[bestTrainMode]['config'])
    bestBackboneStudy["architecture"]["backbone"] = backboneConfigs
    bestBackboneHistories = utils.researchStudies(trainDs, valDs, {'Backbone': bestBackboneStudy}, epochsSearch, verbose)
    bestBackboneHistories['Backbone'+"_"+defaultBackbone['backbone'].__name__] = trainingModeHistories[bestTrainMode]
    bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
    utils.displayStudiesProgress(bestBackboneHistories, path, 'Comparison DNN Backbone')
    utils.saveStudyHistory({'BestBackbone': bestBackboneStudy}, bestBackboneHistories, outFilePath)
    print("Best DNN Backbone:", bestBackboneHistories[bestBackbone]['config']["architecture"]["backbone"].name)


    # --------------------------------------------
    # ---- Stage 3 - Compare FC Classifiers ----
    # --------------------------------------------
    classifierStudy = copy.deepcopy(bestBackboneHistories[bestBackbone]['config'])
    classifierStudy['architecture']["classifier"] = [classifier(nbClasses, "softmax") for classifier in classifiers]
    classifierHistories = utils.researchStudies(trainDs, valDs, {'Classifiers': classifierStudy}, epochsSearch, verbose)
    classifierHistories['Classifier'+"_"+str(defaultClassifier.__name__)] = bestBackboneHistories[bestBackbone]
    bestClassifier = utils.getBestStudyFromHistories(classifierHistories)
    utils.displayStudiesProgress(classifierHistories, path, 'Comparison Classifiers')
    utils.saveStudyHistory({'bestClassifier': classifierStudy}, classifierHistories, outFilePath)
    print("Best Classifier:", str(classifierHistories[bestClassifier]['config']['architecture']["classifier"]))
    
    # ------------------------------------------
    # ---- Stage 4 - Compare Loss Functions ----
    # ------------------------------------------
    lossStudy = copy.deepcopy(classifierHistories[bestClassifier]['config'])
    lossStudy['hyperparameters']["loss"] = losses
    lossesHistories = utils.researchStudies(trainDs, valDs, {'LossFunction': lossStudy}, epochsSearch, verbose)
    lossesHistories['LossFunction'+"_"+str(defaultHypers["loss"])] = classifierHistories[bestClassifier]
    bestLoss = utils.getBestStudyFromHistories(lossesHistories)
    utils.displayStudiesProgress(lossesHistories, path, 'Comparison Loss Functions')
    utils.saveStudyHistory({'BestLoss': lossStudy}, lossesHistories, outFilePath)
    print("Best Loss Function:", str(lossesHistories[bestLoss]['config']['hyperparameters']["loss"]))
    
    
    # ------------------------------------------
    # ---- Stage 5 - Compare Learning Rates ----
    # ------------------------------------------
    learningRatesStudy = copy.deepcopy(lossesHistories[bestLoss]['config'])
    learningRatesStudy['hyperparameters']['learningRate'] = learningRates
    learningRatesHistories = utils.researchStudies(trainDs, valDs, {'LearningRate': learningRatesStudy}, epochsSearch, verbose)
    learningRatesHistories['LearningRate'+"_"+str(defaultHypers["learningRate"])] = lossesHistories[bestLoss]
    bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
    utils.displayStudiesProgress(learningRatesHistories, path, 'Comparison Learning Rate')
    utils.saveStudyHistory({'BestLearningRate': learningRatesStudy}, learningRatesHistories, outFilePath)
    print("Best Learning Rate:", str(learningRatesHistories[bestLearningRate]['config']["hyperparameters"]["learningRate"]))


    # --------------------------------------
    # ---- Stage 6 - Compare Optimizers ----
    # --------------------------------------
    optimizersStudy = copy.deepcopy(learningRatesHistories[bestLearningRate]['config'])
    optimizersStudy["hyperparameters"]["optimizer"] = optimizers
    optimizersHistories = utils.researchStudies(trainDs, valDs, {'Optimizer' : optimizersStudy}, epochsSearch, verbose)
    optimizersHistories['Optimizer'+"_"+defaultHypers['optimizer'].__name__] = learningRatesHistories[bestLearningRate]
    bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
    utils.displayStudiesProgress(optimizersHistories, path, 'Comparison Optimizer')
    utils.saveStudyHistory({'BestOptimizier': optimizersStudy}, optimizersHistories, outFilePath)
    print("Best Optimizier:", optimizersHistories[bestOptimizier]['config']["hyperparameters"]["optimizer"].__name__)
    
    
    
    
    # ----------------------------------------------------------------
    # ---- Stage 7 - Train top K Best models on train+validation -----
    # ----------------------------------------------------------------
    
    histories = {
        **trainingModeHistories, 
        **bestBackboneHistories, 
        **classifierHistories, 
        **lossesHistories,
        **learningRatesHistories,
        **optimizersHistories
    }
    
    
    Path(topModelsPath).mkdir(parents=True, exist_ok=True)
    topKmodelsNames = utils.getBestStudyFromHistories(histories, top)
    topModels = {x: histories.get(x, None) for x in topKmodelsNames}
    topModelsTrainedHistories = {}
    for modelName, model in topModels.items():
        history = model['model'].fit(trainDs.concatenate(valDs), testDs, epochsBest, verbose=1)
        model['model'].save(topModelsPath+modelName)
        topModelsTrainedHistories[modelName] = {
            "history":history,
            "model": model['model'],
            "config": model['config']
        }
    
    utils.saveStudyHistory({'Best Models': None}, topModelsTrainedHistories, topModelsPath+'topHistories.csv', True)
    utils.displayStudiesProgress(topModelsTrainedHistories, topModelsPath)
    return topModelsTrainedHistories
    


In [6]:
def researchStudy(study, path=None):
    datasets = study['datasets']
    studyParams = study['studyHyperParameters']
    preparedDatasets = utils.prepareDatasetsForStudy(datasets, path)
    bestModels = {}
    for dsName, dsObj in preparedDatasets.items():
        bestModels[dsName] = researchBestModel(**dsObj, **studyParams)
    return bestModels


##### **Datasets Configuration**

In [7]:
datasetsConfig = {
    'batchSize': 32, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': (224, 224, 3), 
    'seed': 9, 
    'shuffle': True
    }

#### **2. Gender** 
##### **Datasets:**
* **SOCOFing**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *binary-crossentropy, mean-squared-error, hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:

genderStudy = {
    'datasets' : [
        ds.SOCOFingGender(**datasetsConfig, ds.SOCOFingGender.UNDER_SAMPLING), 
        ds.NISTSDB4Fingers(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'binary_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}

  
genderBestModels = researchStudy(genderStudy, dbStudyOutPath.format('GenderStudy/{}'))

#### **2. Finger Name** 
##### **Datasets:**
* **SOCOFing**
* **NIST Special Database 4**
* **NIST Special Database 300a Roll**
* **NIST Special Database 300b - All Scanners**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *categorical-crossentropy, mean-squared-error, categorical-hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [8]:

fingerNameStudy = {
    'datasets' : [
        ds.SOCOFingFingers(**datasetsConfig), 
        ds.NISTSDB4Fingers(**datasetsConfig), 
        ds.NISTSDB300aFingers(**datasetsConfig), 
        ds.NISTSDB302bFingers(**datasetsConfig)
    ][:2],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'categorical_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'categorical_hinge'][:1],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001][:1],
        'optimizers': [Nadam, RMSprop][:1],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier][:1],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ][:1],
        'epochsSearch': 2,
        'epochsBest': 2,
        'top': 2,
        'verbose': 1
    }
}

  
fingersBestModels = researchStudy(fingerNameStudy, dbStudyOutPath.format('FingerName/{}'))

Metal device set to: AMD Radeon Pro 5500M

systemMemory: 16.00 GB
maxCacheSize: 1.99 GB

Ablation for SOCOfingFingers
Epoch 1/2
132/132 [==============================] - 16s 101ms/step - loss: 2.5270 - accuracy: 0.1036 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/2
132/132 [==============================] - 12s 88ms/step - loss: 2.3050 - accuracy: 0.0964 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 1/2
132/132 [==============================] - 42s 277ms/step - loss: 2.4679 - accuracy: 0.1510 - val_loss: 2.4752 - val_accuracy: 0.1000
Epoch 2/2
132/132 [==============================] - 35s 266ms/step - loss: 1.7720 - accuracy: 0.3329 - val_loss: 2.7066 - val_accuracy: 0.1000
Epoch 1/2
132/132 [==============================] - 42s 281ms/step - loss: 2.4991 - accuracy: 0.1000 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/2
132/132 [==============================] - 37s 281ms/step - loss: 2.3064 - accuracy: 0.1021 - val_loss: 2.3026 - val_accuracy: 0.1000
Backbone Transfer L

INFO:tensorflow:Assets written to: ./out/FingerName/SOCOfingFingers/FinalModels/PT&Untrainable/assets


INFO:tensorflow:Assets written to: ./out/FingerName/SOCOfingFingers/FinalModels/PT&Untrainable/assets


{'history': <keras.callbacks.History object at 0x171be0a00>, 'model': <FPMLmodule.fpml.FPML object at 0x16aa6d430>, 'config': {'architecture': {'backbone': <FPMLmodule.backbones.mobilenet.MobileNet object at 0x167525be0>, 'classifier': <FPMLmodule.classifiers.default.DefaultClassifier object at 0x167c9cac0>, 'inputLayer': None, 'inputDim': (224, 224, 3)}, 'hyperparameters': {'optimizer': <class 'keras.optimizer_v2.adam.Adam'>, 'learningRate': 0.001, 'loss': 'categorical_crossentropy', 'metrics': 'accuracy'}}}
Epoch 1/2
161/161 [==============================] - 44s 269ms/step - loss: 1.4871 - accuracy: 0.4335 - val_loss: 2.4682 - val_accuracy: 0.1000
Epoch 2/2
161/161 [==============================] - 48s 300ms/step - loss: 1.3639 - accuracy: 0.5061 - val_loss: 2.3164 - val_accuracy: 0.1411


INFO:tensorflow:Assets written to: ./out/FingerName/SOCOfingFingers/FinalModels/PT&Trainable/assets


...

INFO:tensorflow:Assets written to: ./out/FingerName/NISTSDB4Fingers/FinalModels/PT&Untrainable/assets


{'history': <keras.callbacks.History object at 0x1748d8df0>, 'model': <FPMLmodule.fpml.FPML object at 0x18fb69310>, 'config': {'architecture': {'backbone': <FPMLmodule.backbones.mobilenet.MobileNet object at 0x174e7b7f0>, 'classifier': <FPMLmodule.classifiers.default.DefaultClassifier object at 0x17f764ee0>, 'inputLayer': None, 'inputDim': (224, 224, 3)}, 'hyperparameters': {'optimizer': <class 'keras.optimizer_v2.adam.Adam'>, 'learningRate': 0.001, 'loss': 'categorical_crossentropy', 'metrics': 'accuracy'}}}
Epoch 1/2
107/107 [==============================] - 35s 326ms/step - loss: 2.3060 - accuracy: 0.1085 - val_loss: 2.3010 - val_accuracy: 0.1217
Epoch 2/2
107/107 [==============================] - 30s 283ms/step - loss: 2.3029 - accuracy: 0.1082 - val_loss: 2.3003 - val_accuracy: 0.1217


INFO:tensorflow:Assets written to: ./out/FingerName/NISTSDB4Fingers/FinalModels/IN&Trainable/assets


INFO:tensorflow:Assets written to: ./out/FingerName/NISTSDB4Fingers/FinalModels/IN&Trainable/assets


{'PT&Untrainable': {'history': <keras.callbacks.History object at 0x1baac1fa0>, 'model': <FPMLmodule.fpml.FPML object at 0x1800531c0>, 'config': {'architecture': {'backbone': <FPMLmodule.backbones.mobilenet.MobileNet object at 0x18df4fd90>, 'classifier': <FPMLmodule.classifiers.default.DefaultClassifier object at 0x180063b20>, 'inputLayer': None, 'inputDim': (224, 224, 3)}, 'hyperparameters': {'optimizer': <class 'keras.optimizer_v2.adam.Adam'>, 'learningRate': 0.001, 'loss': 'categorical_crossentropy', 'metrics': 'accuracy'}}}, 'IN&Trainable': {'history': <keras.callbacks.History object at 0x1bb461640>, 'model': <FPMLmodule.fpml.FPML object at 0x18fb69310>, 'config': {'architecture': {'backbone': <FPMLmodule.backbones.mobilenet.MobileNet object at 0x174e7b7f0>, 'classifier': <FPMLmodule.classifiers.default.DefaultClassifier object at 0x17f764ee0>, 'inputLayer': None, 'inputDim': (224, 224, 3)}, 'hyperparameters': {'optimizer': <class 'keras.optimizer_v2.adam.Adam'>, 'learningRate': 0.

In [9]:
for x, y in fingersBestModels.items():
    for z, w in y.items():
        print(x, w['model'])

SOCOfingFingers <FPMLmodule.fpml.FPML object at 0x16753ef70>
SOCOfingFingers <FPMLmodule.fpml.FPML object at 0x16aa6d430>
NISTSDB4Fingers <FPMLmodule.fpml.FPML object at 0x1800531c0>
NISTSDB4Fingers <FPMLmodule.fpml.FPML object at 0x18fb69310>


#### **3. Fingerprint type** 
##### **Datasets:**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *categorical-crossentropy, mean-squared-error, categorical-hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:

fingerprintTypeNameStudy = {
    'datasets' : [
        ds.NISTSDB4Type(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'categorical_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'categorical_hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}


  
fingerprintTypeBestModels = researchStudy(fingerprintTypeNameStudy, dbStudyOutPath.format('FingerprintType/{}'))

#### **4. Same Person** 
##### **Datasets:**
* **NIST Special Database 4**

##### **Hyperparameters**
* Optimizers: *Adam, Nadam, RMSprop*
* Loss Functions: *binary-crossentropy, mean-squared-error, hinge*
* Learning Rates: *0.1, 0.01, 0.001, 0.0001, 0.00001*
* Backbones: *MobileNetV2, ResNet50, EfficientNetB2, InceptionV3, Xception*

In [ ]:

samePersonStudy = {
    'datasets' : [
        ds.SOCOFingSamePerson(**datasetsConfig), 
        ds.NISTSDB300aSamePerson(**datasetsConfig), 
        ds.NISTSDB302bSamePerson(**datasetsConfig), 
    ],
    'studyHyperParameters' : {
        'defaultBackbone': {'backbone' : bb.MobileNetV2, 'weights' : weightsMNV2},
        'defaultClassifier': cf.DefaultClassifier,
        'defaultHypers': {
            "optimizer": Adam,
            "learningRate": 0.001,
            "loss": 'binary_crossentropy',
            "metrics": 'accuracy'
        },
        'losses': ['mean_squared_error', 'hinge'],
        'learningRates': [0.1, 0.01, 0.0001, 0.00001],
        'optimizers': [Nadam, RMSprop],
        'classifiers': [cf.AlexNetClassifier, cf.MobileNetClassifier, cf.ResNetClassifier],
        'backbonesForSearch': [
            {'backbone' : bb.ResNet50, 'weights' : weightsRN50},
            {'backbone' : bb.EfficientNetB2, 'weights' : weightsENB2},
            {'backbone' : bb.InceptionV3, 'weights' : weightsINCEPTIONV3},
            {'backbone' : bb.Xception, 'weights' : weightsXCEPTION}, 
        ],
        'epochsSearch': 10,
        'epochsBest': 20,
        'top': 5,
        'verbose': 1
    }
}


  
samePersonBestModels = researchStudy(samePersonStudy, dbStudyOutPath.format('SamePerson/{}'))

In [ ]:
fingersBestModels

In [ ]:
def evaluteStudyOnOthersDatasets(bestModels, datasets, epochsFinal, verbose=1, path=None):
    
    evaluations = {}
    for datasetName, bestDatasetModels in bestModels.items():
        chainedTrain = None
        chainedTest = None
        for dset in datasets:
            dset.split = [1]
            [toConcat] = dset.create()
            if(dset.name != datasetName):
                if chainedTrain == None:
                    chainedTrain = toConcat
                else:
                    chainedTrain = chainedTrain.concatenate(toConcat)
            else:
                if chainedTest == None:
                    chainedTest = toConcat
                else:
                    chainedTest.concatenate(toConcat)
        evaluations[datasetName] = {}
        for modelName, model in bestDatasetModels.items():
            evaluation = model['model'].getModel().evaluate(chainedTrain)
            evaluations[datasetName][modelName] = {
                'evaluation': evaluation,
                'train' : chainedTrain, 
                'test': chainedTest
            }
    return evaluations    

    # finalModelConfiguration = bestModels[bestModelName][1]
    # finalModel = FPML(**finalModelConfiguration["architecture"]).create(**finalModelConfiguration["hyperparameters"])
    # finalModelHistory = finalModel.fit(finalDs['train'], validation_data=finalDs['test'], epochs=epochsFinal, verbose=verbose)
    
    # utils.datasetAnalysisAndDisplay([finalDs['train'], finalDs['test']], datasets[0], path)
    
    # toSave = {'Final Model':{'history' : finalModelHistory, 'config' : finalModelConfiguration}}
    
    # finalModel.save(path+'FinalModel-'+bestModelName)
    # utils.displayStudiesProgress(toSave, path, 'Final Model')
    # utils.saveStudyHistory({'Final Model': finalModelConfiguration}, toSave, path+'finalModel.csv')
    # utils.displayConfusion(finalDs['test'], finalModel, path)


evaluations = evaluteStudyOnOthersDatasets(fingersBestModels, fingerNameStudy['datasets'], 2, 1)

In [ ]:
for key, val in fingersBestModels.items():
    for key2, val2 in val.items():
        print(val2['model'].getModel())